# Long Short Term Memory

In [339]:
import csv
import math
import re
from abc import abstractmethod, ABC
from collections import Counter

import numpy as np

np.random.seed(99)

## Foundation

### Tensor

In [340]:
class Tensor:

    def __init__(self, data):
        self.data = np.array(data)
        self.grad = None
        self.gradient_fn = lambda: None
        self.parents = set()

    def backward(self):
        if self.gradient_fn:
            self.gradient_fn()

        for p in self.parents:
            p.backward()

    def shape(self):
        return self.data.shape

    def size(self):
        return np.prod(self.data.shape[1:])

    def __str__(self):
        return str(self.data)

    def __add__(self, other):
        p = Tensor(self.data + other.data)

        def gradient_fn():
            self.grad = p.grad
            other.grad = p.grad

        p.gradient_fn = gradient_fn
        p.parents = {self, other}
        return p

    def __mul__(self, other):
        p = Tensor(self.data * other.data)

        def gradient_fn():
            self.grad = p.grad * other.data
            other.grad = p.grad * self.data

        p.gradient_fn = gradient_fn
        p.parents = {self, other}
        return p

    def concat(self, other, axis):
        p = Tensor(np.concatenate([self.data, other.data], axis=axis))

        def gradient_fn():
            self.grad, other.grad = np.split(p.grad, [self.data.shape[axis]], axis=axis)

        p.gradient_fn = gradient_fn
        p.parents = {self, other}
        return p

### Base Dataset

In [341]:
class Dataset(ABC):

    def __init__(self, batch_size=1):
        self.batch_size = batch_size
        self.load()
        self.train()

    @abstractmethod
    def load(self):
        pass

    def train(self):
        self.features = self.train_features
        self.labels = self.train_labels

    def eval(self):
        self.features = self.test_features
        self.labels = self.test_labels

    def shape(self):
        return Tensor(self.features).size(), Tensor(self.labels).size()

    def items(self):
        return Tensor(self.features), Tensor(self.labels)

    def __len__(self):
        return len(self.features) // self.batch_size

    def __getitem__(self, index):
        start = index * self.batch_size
        end = start + self.batch_size
        return Tensor(self.features[start: end]), Tensor(self.labels[start: end])

    @abstractmethod
    def estimate(self, predictions):
        pass

### Base Layer

In [342]:
class Layer(ABC):

    def __init__(self):
        self.training = True

    def __call__(self, *args):
        return self.forward(*args)

    def train(self):
        self.training = True

    def eval(self):
        self.training = False

    @abstractmethod
    def forward(self, *args):
        pass

    def parameters(self):
        return []

    def __str__(self):
        return ''

### Base Loss Function

In [343]:
class Loss(ABC):

    @abstractmethod
    def __call__(self, p: Tensor, y: Tensor):
        pass

### Base Optimizer

In [344]:
class Optimizer(ABC):

    def __init__(self, parameters, lr):
        self.parameters = parameters
        self.lr = lr

    @abstractmethod
    def backward(self):
        pass

### Base Model

In [345]:
class Model(ABC):

    def __init__(self, layer, loss, optimizer):
        self.layer = layer
        self.loss = loss
        self.optimizer = optimizer

    @abstractmethod
    def train(self, dataset, epochs):
        pass

    @abstractmethod
    def test(self, dataset):
        pass

## Data

### RNN Dataset

In [346]:
class RNNDataset(Dataset):

    def __init__(self, filename):
        self.filename = filename
        super().__init__()

    def load(self):
        self.reviews = []
        self.sentiments = []
        with open(self.filename, 'r', encoding='utf-8') as f:
            reader = csv.reader(f)
            next(reader)
            for _, row in enumerate(reader):
                self.reviews.append(row[0])
                self.sentiments.append(row[1])

        split_reviews = []
        for line in self.reviews:
            split_reviews.append(self.clean_text(line.lower()).split())

        self.vocabulary = set(word for line in split_reviews for word in line)
        self.word2index = {word: index for index, word in enumerate(self.vocabulary)}
        self.index2word = {index: word for index, word in enumerate(self.vocabulary)}
        self.tokens = [[self.word2index[word] for word in line if word in self.word2index] for line in split_reviews]

    @staticmethod
    def clean_text(text):
        text = re.sub(r'<[^>]+>', '', text)
        text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
        return text

    def train(self):
        self.features = []
        self.labels = []
        for line in self.tokens[:-10]:
            for index in range(len(line) - 4):
                self.features.append(line[index: index + 4])
                self.labels.append(self.onehot(line[index + 4]))
    def eval(self):
        self.features = []
        self.labels = []
        for line in self.tokens[-10:]:
            for index in range(len(line) - 4):
                self.features.append(line[index: index + 4])
                self.labels.append(self.onehot(line[index + 4]))

    def encode(self, text):
        words = self.clean_text(text.lower()).split()
        return [self.word2index[word] for word in words]

    def decode(self, tokens):
        return " ".join([self.index2word[index] for index in tokens])

    def onehot(self, token):
        ebd = np.zeros(len(self.vocabulary))
        ebd[token] = 1
        return ebd

    @staticmethod
    def argmax(vector):
        return [np.argmax(vector)]

    def estimate(self, predictions):
        count = 0
        for i in range(len(predictions)):
            if self.argmax(predictions[i].data[0]) == self.argmax(self.labels[i].data):
                count += 1
        return count / len(predictions)

## Model

### Linear Layer

In [347]:
class Linear(Layer):

    def __init__(self, in_size, out_size):
        super().__init__()
        self.weight = Tensor(np.random.rand(out_size, in_size) / in_size)
        self.bias = Tensor(np.zeros(out_size))

    def forward(self, x: Tensor):
        p = Tensor(x.data @ self.weight.data.T + self.bias.data)

        def gradient_fn():
            self.weight.grad = p.grad.T @ x.data / len(x.data)
            self.bias.grad = np.sum(p.grad, axis=0) / len(x.data)
            x.grad = p.grad @ self.weight.data

        p.gradient_fn = gradient_fn
        p.parents = {self.weight, self.bias, x}
        return p

    def parameters(self):
        return [self.weight, self.bias]

    def __str__(self):
        return f'weight: {self.weight}\nbias: {self.bias}'

### Sequential Layer

In [348]:
class Sequential(Layer):

    def __init__(self, layers):
        super().__init__()
        self.layers = layers

    def train(self):
        for l in self.layers:
            l.train()

    def eval(self):
        for l in self.layers:
            l.eval()

    def forward(self, x: Tensor):
        for l in self.layers:
            x = l(x)
        return x

    def parameters(self):
        return [p for l in self.layers for p in l.parameters()]

    def __str__(self):
        return '\n'.join(str(l) for l in self.layers if str(l))

### Embedding Layer

In [349]:
class Embedding(Layer):

    def __init__(self, vocabulary_size, embedding_size, axis=1):
        super().__init__()
        self.vocabulary_size = vocabulary_size
        self.embedding_size = embedding_size
        self.axis = axis

        self.weight = Tensor(np.random.rand(embedding_size, vocabulary_size) / vocabulary_size)

    def forward(self, x: Tensor):
        p = Tensor(np.sum(self.weight.data.T[x.data], axis=self.axis))

        def gradient_fn():
            if self.weight.grad is None:
                self.weight.grad = np.zeros_like(self.weight.data)
            self.weight.grad.T[x.data] += p.grad

        p.gradient_fn = gradient_fn
        p.parents = {self.weight}
        return p

    def parameters(self):
        return [self.weight]

### ReLU Activation Function

In [350]:
class ReLU(Layer):

    def forward(self, x: Tensor):
        p = Tensor(np.maximum(0, x.data))

        def gradient_fn():
            x.grad = (p.data > 0) * p.grad

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

### Tanh Activation Function

In [351]:
class Tanh(Layer):

    def forward(self, x: Tensor):
        p = Tensor(np.tanh(x.data))

        def gradient_fn():
            x.grad = p.grad * (1 - p.data ** 2)

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

### Sigmoid Activation Function

In [352]:
class Sigmoid(Layer):

    def __init__(self, clip_range=(-100, 100)):
        super().__init__()
        self.clip_range = clip_range

    def forward(self, x: Tensor):
        z = np.clip(x.data, self.clip_range[0], self.clip_range[1])
        p = Tensor(1 / (1 + np.exp(-z)))

        def gradient_fn():
            x.grad = p.grad * p.data * (1 - p.data)

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

### Softmax Activation Function

In [353]:
class Softmax(Layer):

    def __init__(self, axis=1):
        super().__init__()
        self.axis = axis

    def forward(self, x: Tensor):
        exp = np.exp(x.data - np.max(x.data, axis=self.axis, keepdims=True))
        p = Tensor(exp / np.sum(exp, axis=self.axis, keepdims=True))

        def gradient_fn():
            x.grad = np.zeros_like(x.data)
            for idx in range(x.data.shape[0]):
                itm = p.data[idx].reshape(-1, 1)
                x.grad[idx] = (np.diagflat(itm) - itm @ itm.T) @ p.grad[idx]

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

## LSTM Layer

In [354]:
class LSTM(Layer):

    def __init__(self, vocabulary_size, embedding_size):
        super().__init__()
        self.vocabulary_size = vocabulary_size
        self.embedding_size = embedding_size

        self.embedding = Embedding(vocabulary_size, embedding_size)
        self.forget_gate = Linear(embedding_size * 2, embedding_size)
        self.input_gate = Linear(embedding_size * 2, embedding_size)
        self.output_gate = Linear(embedding_size * 2, embedding_size)
        self.cell_update = Linear(embedding_size * 2, embedding_size)
        self.output = Linear(embedding_size, vocabulary_size)
        self.sigmoid = Sigmoid()
        self.tanh = Tanh()

        self.layers = [self.embedding,
                       self.forget_gate,
                       self.input_gate,
                       self.output_gate,
                       self.cell_update,
                       self.output,
                       self.sigmoid,
                       self.tanh]

    def __call__(self, x: Tensor, c: Tensor, h: Tensor):
        return self.forward(x, c, h)

    def forward(self, x: Tensor, c: Tensor, h: Tensor):
        if not c:
            c = Tensor(np.zeros((1, self.embedding_size)))
        if not h:
            h = Tensor(np.zeros((1, self.embedding_size)))

        embedding_feature = self.embedding(x)
        concat_feature = self.tanh(embedding_feature.concat(h, axis=1))
        forget_hidden = self.sigmoid(self.forget_gate(concat_feature))
        input_hidden = self.sigmoid(self.input_gate(concat_feature))
        output_hidden = self.sigmoid(self.output_gate(concat_feature))
        cell_hidden = self.tanh(self.cell_update(concat_feature))
        cell_feature = forget_hidden * c + input_hidden * cell_hidden
        hidden_feature = output_hidden * self.tanh(cell_feature)

        return (self.output(hidden_feature),
                Tensor(cell_feature.data),
                Tensor(hidden_feature.data))

    def parameters(self):
        return [p for l in self.layers for p in l.parameters()]

### MSE Loss Function

In [355]:
class MSELoss(Loss):

    def __call__(self, p: Tensor, y: Tensor):
        mse = Tensor(((p.data - y.data) ** 2).mean())

        def gradient_fn():
            p.grad = (p.data - y.data) * 2

        mse.gradient_fn = gradient_fn
        mse.parents = {p}
        return mse

### Cross Entropy Loss Function

In [356]:
class CELoss:

    def __call__(self, p: Tensor, y: Tensor):
        exp = np.exp(p.data - np.max(p.data, axis=-1, keepdims=True))
        softmax = exp / np.sum(exp, axis=-1, keepdims=True)

        log = np.log(np.clip(softmax, 1e-10, 1))
        ce = Tensor(0 - np.sum(y.data * log) / len(p.data))

        def gradient_fn():
            p.grad = (softmax - y.data) / len(p.data)

        ce.gradient_fn = gradient_fn
        ce.parents = {p}
        return ce

### Binary Cross Entropy Loss Function

In [357]:
class BCELoss:

    def __call__(self, p: Tensor, y: Tensor):
        clipped = np.clip(p.data, 1e-7, 1 - 1e-7)
        bce = Tensor(-np.mean(y.data * np.log(clipped)
                              + (1 - y.data) * np.log(1 - clipped)))

        def gradient_fn():
            p.grad = (clipped - y.data) / (clipped * (1 - clipped)) * len(p.data)

        bce.gradient_fn = gradient_fn
        bce.parents = {p}
        return bce

### SGD Optimizer

In [358]:
class SGDOptimizer(Optimizer):

    def backward(self):
        for p in self.parameters:
            p.data -= p.grad * self.lr

### LSTM Neural Network Model

In [359]:
class LSTMModel(Model):

    def train(self, dataset, epochs):
        layer.train()
        dataset.train()

        for epoch in range(epochs):
            cell = hidden = None
            for i in range(len(dataset)):
                feature, label = dataset[i]

                prediction, cell, hidden = self.layer(feature,
                                                      cell,
                                                      hidden)
                error = self.loss(prediction,
                                  label)

                error.backward()
                self.optimizer.backward()

    def test(self, dataset):
        layer.eval()
        dataset.eval()

        predictions = []
        cell = hidden = None
        for i in range(len(dataset)):
            feature, label = dataset[i]
            prediction, cell, hidden = self.layer(feature,
                                                  cell,
                                                  hidden)
            predictions.append(prediction)
        return predictions

## Configuration

### Learning Rate

In [360]:
LEARNING_RATE = 0.01

### Epoch

In [361]:
EPOCHS = 10

## Training

### Iteration Training

In [362]:
dataset = RNNDataset('reviews.csv')
layer = LSTM(len(dataset.vocabulary), 64)
loss = CELoss()
optimizer = SGDOptimizer(layer.parameters(),
                         lr=LEARNING_RATE)

model = LSTMModel(layer,
                  loss,
                  optimizer)
model.train(dataset,
            EPOCHS)

## Testing

### Estimating

In [363]:
predictions = model.test(dataset)

print(f'Accuracy: {dataset.estimate(predictions)}')

Accuracy: 0.14012738853503184


### Comparing Labels and Predictions

In [364]:
features, labels = dataset.items()
for i in range(len(predictions)):
    pos = np.argmax(predictions[i].data[0])
    print(f'Feature: {dataset.decode(features.data[i])} | '
          f'Label: {dataset.decode(dataset.argmax(labels.data[i]))} | '
          f'Prediction: {dataset.decode(dataset.argmax(predictions[i].data[0]))}')

Feature: worst movie with awful | Label: music | Prediction: the
Feature: movie with awful music | Label: the | Prediction: the
Feature: with awful music the | Label: actor | Prediction: the
Feature: awful music the actor | Label: did | Prediction: the
Feature: music the actor did | Label: a | Prediction: the
Feature: the actor did a | Label: boring | Prediction: the
Feature: actor did a boring | Label: job | Prediction: the
Feature: did a boring job | Label: actress | Prediction: the
Feature: a boring job actress | Label: director | Prediction: the
Feature: boring job actress director | Label: character | Prediction: the
Feature: job actress director character | Label: screenplay | Prediction: actress
Feature: actress director character screenplay | Label: scene | Prediction: the
Feature: director character screenplay scene | Label: but | Prediction: by
Feature: character screenplay scene but | Label: or | Prediction: by
Feature: screenplay scene but or | Label: it | Prediction: by
Fe